In [18]:
from pypalace import Simulation, Config, Model, Domains, Boundaries, Solver

## Eigenmode simulation

 ##### First let's create the config object (automatically creates config["Problem"] and also define config["Model"]

In [19]:
meshfile = "qubit_resonator_mesh.bdf"
my_sim = Config("Eigenmode",Output="eigenmode_output") # config["Problem"]
 
# define adaptive mesh refinement, the qubit is coarsely meshed to save on file size so we use AMR to boost simulation accuracy
my_refinement = Model.Refinement(Tol = 1e-6,MaxIts = 4)

# define config["Model"] block
my_sim.add_Model(meshfile,L0=1e-6,Refinement=my_refinement)

##### Before we start defining our materials and boundary conditions, let's take a look at the blocks defined in our meshfile

In [20]:
my_attributes = Simulation.get_mesh_attributes(meshfile)
my_attributes

,Name,ID,Type
0,substrate,1,Volume
1,air,2,Volume
2,feedline,3,Surface
3,resonator,4,Surface
4,qubit,5,Surface
5,JJ,6,Surface
6,ground_plane,7,Surface
7,far_field,8,Surface


**Volumes**
* we'll define substrate (1) as sapphire
* and we'll define air as a vacuum

**Surfaces**
* feedline (3), resonator (4), and qubit (5), and ground plane are the superconducting components which we will define as PEC
* JJ is the Josephson junction, we will define this as a lumped port with inductance $L_J$ = 10.4 nH
* far field is the geometry closing boundary, we also define this as PEC, some folk define it as an absorbing boundary condition instead.

##### Define our materials and boundary conditions

In [21]:
# define materials
sapphire = Domains.Material(Attributes = [1],Permeability=1.0,Permittivity=[9.3,9.3,11.4],MaterialAxes=[[1,0,0],[0,1,0],[0,0,1]],LossTan=8.6e-5)
air = Domains.Material([2],1.0,1.0,0.0)
my_materials = [sapphire,air] # material list for input into add_Domains()

# define boundary conditions
PECs = Boundaries.PEC([3,4,5,7,8]) # 
JJ = Boundaries.LumpedPort(Index=1,Attributes=[6],Direction="+X",R=0,L=round(10.4*10**(-9),9),C=0) 
my_BCs = [PECs,JJ] # boundary condition list for input into add_Boundaries()

# add config["Domains"] and config["Boundaries"] using our material and BC lists above
my_sim.add_Domains(my_materials)
my_sim.add_Boundaries(my_BCs)

##### Define eigenmode and linear solver parameters

In [22]:
## eigenmode parameters
eigenmode_params = Solver.Eigenmode(Target = 3.0,
                                    Tol = 1.0e-8,
                                    N = 6,
                                    Save = 6)
## linear solver parameters
Linear_params = Solver.Linear(Type="Default",
                              KSPType = "Default",
                              Tol = 1e-8,
                              MaxIts = 50)

## add them to config["Solver"] and solver["Linear"]
my_sim.add_Solver(Simulation=eigenmode_params,Order = 2,Linear=Linear_params)

##### Check the config to make sure it looks good

In [23]:
my_sim.print_config()

{
  "Problem": {
    "Type": "Eigenmode",
    "Verbose": 2,
    "Output": "eigenmode_output"
  },
  "Model": {
    "Mesh": "qubit_resonator_mesh.bdf",
    "L0": 1e-06,
    "Refinement": {
      "Tol": 1e-06,
      "MaxIts": 4
    }
  },
  "Domains": {
    "Materials": [
      {
        "Attributes": [
          1
        ],
        "Permeability": 1.0,
        "Permittivity": [
          9.3,
          9.3,
          11.4
        ],
        "LossTan": 8.6e-05,
        "MaterialAxes": [
          [
            1,
            0,
            0
          ],
          [
            0,
            1,
            0
          ],
          [
            0,
            0,
            1
          ]
        ]
      },
      {
        "Attributes": [
          2
        ],
        "Permeability": 1.0,
        "Permittivity": 1.0,
        "LossTan": 0.0
      }
    ]
  },
  "Boundaries": {
    "PEC": {
      "Attributes": [
        3,
        4,
        5,
        7,
        8
      ]
    },
    "Lu

The MaterialAxes definition for sapphire gets a bit ugly in the config file... working to beautify it (:

##### Save the config file

In [24]:
my_sim.save_config("qubit_res.json",check_validity=True) # checks validity of file and raises error if something is missing